# COGS 108 - Data Checkpoint

# Names

- Allen Zou
- Brian Han
- Dillen Padhiar
- Lian Su


# Research Question

Are there certain factors that correlate with frequent alcohol use by high school students in the United States, particularly with personal factors such as sex and age, parental and family factors such as their education, size, income, etc., school performance factors such as workload/free time, educational support, grades, etc.?


# Dataset(s)

- Dataset Name: Student Alcohol Consumption
- Link to the dataset: https://www.kaggle.com/uciml/student-alcohol-consumption
- Number of observations: 395 observations in the math class dataset, 649 observations in the Portuguese class dataset, and 382 overlapping observations

The Student Alcohol Consumption is a 2008 dataset contains survey responses from students in a math and Portuguese class taken in Porto, Portugal. The survey reports alcohol consumption, many personal factors such as sex and age, parental and family factors such as their education level, job type, etc., and school performance factors including those mentioned in our research question.

# Setup

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
# import patsy
# import statsmodels.api as sm
# import os
# import folium
# from folium import plugins
# from folium.plugins import HeatMap

# Data Cleaning

The link to the dataset contains two separate CSV files, one of which contains students from math classes and Portuguese language classes in high schools. It's important to note, however, that there are 382 students that overlap in these two subsets of classes. We load both of these datasets into DataFrames named student_math and student_portuguese.

In [27]:
# Read in the dataset
student_math = pd.read_csv('student-mat.csv')
student_portuguese = pd.read_csv('student-por.csv')

print('The math class dataset contains {} students and {} columns.'.format(len(student_math.index), len(student_math.columns)))
print('The Portuguese class dataset contains {} students and {} columns.'.format(len(student_portuguese.index), len(student_portuguese.columns)))

The math class dataset contains 395 students and 33 columns.
The Portuguese class dataset contains 649 students and 33 columns.


Unfortunately, there aren't that many students in either dataset. However, because there are only 13 students in the math class dataset that don't overlap with those in the Portuguese class dataset, we feel like it would still be fine to drop the student_math DataFrame. That is, it shouldn't affect our final analysis much.

In [28]:
# Since we're now ignoring the math class dataset, lets "rename" the Portuguese dataset to something shorter
student = student_portuguese

# See what columns are in the dataset
print("Columns in the Portuguese dataset: ", end = '')
print(*student_portuguese, sep = ', ')

Columns in the Portuguese dataset: school, sex, age, address, famsize, Pstatus, Medu, Fedu, Mjob, Fjob, reason, guardian, traveltime, studytime, failures, schoolsup, famsup, paid, activities, nursery, higher, internet, romantic, famrel, freetime, goout, Dalc, Walc, health, absences, G1, G2, G3


This dataset contains a ton of information for each student, but for the sake of simplicity for our project, we will only keep the ones that are relevant to our reserach question.

In [29]:
# Drop the irrelevant columns
student = student.loc[:, ['sex', 'age', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'guardian', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']]

We chose to drop the following for these reasons:
- school (Gabriel Pereira or Mousinho da Silveira): we feel that the difference in schools chosen shouldn't impact our final analysis much. Additionally, it's not needed for our research question.
- address (urban or rural): this also isn't relevant for our research question.
- reason (close to home, reputation, course, preference, etc.): same reason as school.
- traveltime (how long it takes to get to school): also doesn't relate to our research question much.
- nursery (binary question of attends nursery school?): same as above

What we noticed is that this dataset is already relatively clean, but we should still check for missing values.

In [30]:
# Are there any NaN entries?
if(student.isnull().values.any()):
    print('There are NaN values in this dataset.')
else:
    print('There are no NaN values in this dataset.')

There are no NaN values in this dataset.


It turns out the creators of this dataset, Fabio Pagnotta and Hossain Mohammad Amran, already pruned missing values.

We also noticed that there are a ton of columns that have strings are entries. To make things simpler, we'll quantify them (more on this after the changes to and renaming of columns). Additionally, some of the names aren't very intuitive. We'll change them after we quantify these non-numeric columns.

In [31]:
# Binary columns
# sex: represent 'M' with 0, 'F' with 1 
student['sex'] = student['sex'].replace({'M': 0, 'F': 1})

# famsize: represent 'LE3' with 0, 'GT3' with 1
student['famsize'] = student['famsize'].replace({'LE3': 0, 'GT3': 1})

# Pstatus: represent 'A' wiht 0, 'T' with 1
student['Pstatus'] = student['Pstatus'].replace({'A': 0, 'T': 1})

# guardian: represent 'mother' with 0, 'father' with 1
student['guardian'] = student['guardian'].replace({'mother': 0, 'father': 1, 'other': 2})

# schoolsup: represent 'no' with 0, 'yes' with 1
student['schoolsup'] = student['schoolsup'].replace({'no': 0, 'yes': 1})

# famsup: represent 'no' with 0, 'yes' with 1
student['famsup'] = student['famsup'].replace({'no': 0, 'yes': 1})

# paid: represent 'no' with 0, 'yes' with 1
student['paid'] = student['paid'].replace({'no': 0, 'yes': 1})

# activities: represent 'no' with 0, 'yes' with 1
student['activities'] = student['activities'].replace({'no': 0, 'yes': 1})

# higher: represent 'no' with 0, 'yes' with 1
student['higher'] = student['higher'].replace({'no': 0, 'yes': 1})

# internet: represent 'no' with 0, 'yes' with 1
student['internet'] = student['internet'].replace({'no': 0, 'yes': 1})

# romantic: represent 'no' with 0, 'yes' with 1
student['romantic'] = student['romantic'].replace({'no': 0, 'yes': 1})


# Non-binary columns
# Mjob: represent 'teacher' with 0, 'health' with 1, 'services' with 2,
#       'at_home' with 3, 'other' with 4
student['Mjob'] = student['Mjob'].replace({'teacher': 0, 'health': 1,
                  'services': 2, 'at_home': 3, 'other': 4})

# Fjob: same as Mjob -- represent 'teacher' with 0, 'health' with 1,
#        'services' with 2, 'at_home' with 3, 'other' with 4
student['Fjob'] = student['Fjob'].replace({'teacher': 0, 'health': 1,
                  'services': 2, 'at_home': 3, 'other': 4})

# Medu: the ordering seems weird -- it's listed as 0: none, 1: primary education,
# 2: 5th - 9th, 3: secondary.
# Let's reorder it so it's 0: none, 1: 5th - 9th, 2: secondary, and 3: primary
# education instead
student['Medu'] = student['Medu'].replace({0: 0, 1: 3, 2: 1, 3: 2})

# Fedu: same as Medu
student['Fedu'] = student['Fedu'].replace({0: 0, 1: 3, 2: 1, 3: 2})

# studytime: keep everything the same, but index by 0 instead because apprently
# the survey used answer choice options with ranges instead of letting the
# students put their own study time
student['studytime'] = student['studytime'] - 1

In [32]:
# Now let's rename some of these columns
student = student.rename(columns = {'Pstatus': 'cohab_status', 'Medu': 'mom_edu',
                                    'Fedu': 'dad_edu', 'Mjob': 'mom_job',
                                    'Fjob': 'dad_job', 'paid': 'extra_paid_classes',
                                    'higher': 'higher_edu', 'famrel': 'fam_rel',
                                    'goout': 'hang_out', 'famsize': 'fam_size',
                                    'studytime': 'study_time', 'freetime': 'free_time',
                                    'schoolsup': 'school_sup', 'famsup': 'fam_sup'})

Some columns aren't changed yet because we will deal with them separately. In case it isn't too clear what changed, we've listed the changes below:
- famsize: fam_size (family size)
- Pstatus: cohab_status (parent's cohabition status; living apart or together?)
- Medu: mom_edu (mother's education level)
- Fedu: dad_edu (father's education level)
- Mjob: mom_job (mother's job)
- Fjob: dad_job (father's job)
- studytime: study_time (how much time the student studies per week)
- schoolsup: school_sup (access to extra educational support)
- famsup: fam_sup (student has family educational support)
- paid: extra_paid_classes (extra paid classes within the Portuguese subject)
- higher: higher_edu (does the student want to pursue higher education?)
- famrel: fam_rel (quality of family relationship)
- freetime: free_time (rating for how much free time a student has after school)
- goout: hang_out (how much they hang out with friends)
   
Lastly before we describe what each column means and how we coded the responses for each of the columns, we want to combine a few columns.

In [ ]:
# Merge Dalc and Walc
student['alcohol_frequency'] = student['Dalc'] + student['Walc']
print("We can see that the a student's maximum alcohol consumption frequency throughout the week is 10, whereas the minimum frequency is 2.")
print(student['alcohol_frequency'].describe())

We can see that the a student's maximum alcohol consumption frequency throughout the week is 10, whereas the minimum frequency is 2.
count    649.000000
mean       3.782743
std        1.992411
min        2.000000
25%        2.000000
50%        3.000000
75%        5.000000
max       10.000000
Name: alcohol_frequency, dtype: float64


Dalc contains responses for student alcohol consumption frequency during weekdays while Walc contains responses for student alcohol consumption frequency during weekends. We decided to add these two frequencies and changing the response scale from 2 - 10.

In [34]:
# G1 and G2 represent the grades for period 1 and period 2, but G3 represents
# the final grade. We want to see how different the average of G1 and G2
# are from G3.
combined = (student['G1'] + student['G2']) / 2
student['difference'] = student['G3'] - combined
print("Let's first get a brief description of these differences.")
print(student['difference'].describe())

Let's first get a brief description of these differences.
count    649.000000
mean       0.421418
std        1.388329
min      -10.000000
25%        0.000000
50%        0.500000
75%        1.000000
max        6.000000
Name: difference, dtype: float64


Before we figure out what to do with the period 1 (G1), period 2 (G2), and final grade columns (G3),
we should see how much of a discrepency there is between the average of the period grades and the
final grade.

In [ ]:
# The IQR looks really good because the deviations aren't that big, but how
# many students are actually in that range?
numStudents = student['difference'][(student['difference'] >= 0) &
                                    (student['difference'] <= 1)].value_counts()
print("Number of students in the IQR")
print(numStudents)

# Notice how the IQR contains positive differences. We should check the other
# direction (-1.0 to 0.0) to see how many students fall in that difference too.
numStudents = student['difference'][(student['difference'] < 0) |
                                    (student['difference'] > 1)].value_counts()
print("Number of students outside the IQR")
print(numStudents)

# This looks promising in terms of pruning, especially because our dataset is
# already pretty small. We think a difference in the range of -2.0 to 2.0
# won't introduce many confounding variables that we aren't aware of or can't
# measure, but how many people are in that range?
numStudents = student['difference'][(student['difference'] >= -2.0) &
                                    (student['difference'] <= 2.0)].shape[0]
print("Number of students within the range [-2.0, 2.0]")
print(numStudents)

print("Number of students we are pruning")
print(student.shape[0] - numStudents)

# We're only pruning 43, which is much better than the ~50% could have pruned
# just from looking at the information from describe(). Let's remove those
# students.
student = student[(student['difference'] >= -2.0) &
                  (student['difference'] <= 2.0)]
student = student.reset_index(drop = True)
student

# Let's drop the period 1 (G1), period 2 (G2), and the difference column and only
# use the final grade (G3) for our analysis. Rename the column to final_grade
# since it makes more sense
student = student.drop(columns = ['G1', 'G2', 'difference'])
student = student.rename(columns = {'G3': 'final_grade'})

# Drop Dalc and Walc because we combined them into alcohol_frequency
student = student.drop(columns = ['Walc', 'Dalc'])

Number of students in the IQR
0.5    157
0.0    121
1.0     98
Name: difference, dtype: int64
Number of students outside the IQR
 1.5     93
-0.5     83
 2.0     27
-1.0     21
 2.5     17
 3.0      5
-1.5      4
-2.5      4
-7.0      3
-2.0      2
-3.5      2
 6.0      1
-3.0      1
 3.5      1
-10.0     1
-9.0      1
-6.5      1
-7.5      1
-8.0      1
-4.5      1
-4.0      1
-6.0      1
 5.5      1
Name: difference, dtype: int64
Number of students within the range [-2.0, 2.0]
606
Number of students we are pruning
43


There is quite a lot of code this time, but to briefly describe what we did, we noticed that there were only 43 students in the dataset had a difference in grade larger than 2. Although the grading
scale is quite different in this school (out of 20 instead of the usual 100 in most schools in the United States), we thought that a difference within [-2.0, 2.0] wouldn't affect our analyses much.
Our reasoning for keeping this constraint is because based on our research question, we aren't trying
to analyze change in behavior over time.

In [ ]:
# One last thing, the absenses column is the only column that either doesn't need to be coded or has a
# integer value response restricted to a range. Lets see what the values are in absences before
# writing out the description of each column.
print("Minimum number of absences: " + str(student['absences'].min()))
print("Maximum number of absences: " + str(student['absences'].max()))

Minimum number of absences: 0
Maximum number of absences: 32


We won't be adding or removing columns past this point, so we'll describe what each of the remaining columns are and how they're coded (if we had to change them):
- sex: sex of the student
    - Male: 0
    - Female: 1
- age: age of the student
    - numeric value
- fam_size: indicates if the family size if less than or equal to 3 or more than 3
    - less than or equal to 3 (LE3): 0
    - greater than 3 (GT3): 1
- cohab_status: indicates if the family is living together or away
    - away (A): 0
    - together (T): 1
- mom_edu: mother's level of education
    - none: 0
    - 5th - 9th: 1
    - secondary: 2
    - primary: 3
- dad_edu: father's level of education
    - none: 0
    - 5th - 9th: 1
    - secondary: 2
    - primary: 3
- mom_job: mother's type of job
    - teacher: 0
    - health: 1
    - services: 2
    - at_home: 3
    - other: 4
- dad_job: father's type of job
    - teacher: 0
    - health: 1
    - services: 2
    - at_home: 3
    - other: 4
- guardian: student's guardian
    - mother: 0
    - father: 1
    - other: 2
- study_time: student's weekly study time
    - less than 2 hours: 0
    - 2-5 hours: 1
    - 5-10 hours: 2
    - greater than 10 hours: 3
- failures: student's number of past failed classes
    - numeric value, except it caps at 4
- school_sup: indicates if the student got extra educational support
    - no: 0
    - yes: 1
- fam_sup: indicates if the student recieved family educational support
    - no: 0
    - yes: 1
- extra_paid: indicates if the student has extra paid classes within the Portuguese subject
    - no: 0
    - yes: 1
- activities: indicates if the student participates in extra-cirricular activies
    - no: 0
    - yes: 1
- schoolsup: indicates if the student got extra educational support
    - no: 0
    - yes: 1
- higher_edu: indicates if the student wants to take higher education
    - no: 0
    - yes: 1
- internet: indicates if the student has access to internet at home
    - no: 0
    - yes: 1
- romantic: indicates if the student is currently in a romantic relationship
    - no: 0
    - yes: 1
- fam_rel: student's quality of family relationship
    - numeric value indicating a rating from 1 to 5
- free_time: student's amount of free time
    - numeric value indicating a rating from 1 to 5
- hang_out: how often the student hangs out with friends
    - numeric value indicating a rating from 1 to 5
- alcohol_frequency: how often the student drinks alcohol
    - numeric value indicating a summed rating from 2 - 10
- health: health rating of the student
    - numeric value indicating a rating from 1 to 5
- absences: how many absences the student has
    - numeric value (seems to range from 0 to 32, although the
      data set says it can go up to 93)
- final_grade: the final grade of the student
    - numeric value from 0 to 20
    

Now that it's clear what each column is for, we should perform a few more checks for clean up.

In [ ]:
# Check if everything is actually quantified
student.dtypes

sex                   int64
age                   int64
fam_size              int64
cohab_status          int64
mom_edu               int64
dad_edu               int64
mom_job               int64
dad_job               int64
guardian              int64
study_time            int64
failures              int64
school_sup            int64
fam_sup               int64
extra_paid_classes    int64
activities            int64
higher_edu            int64
internet              int64
romantic              int64
fam_rel               int64
free_time             int64
hang_out              int64
health                int64
absences              int64
final_grade           int64
alcohol_frequency     int64
dtype: object

In [ ]:
# Now check if the values for each column are all in range
# We can skip the ones that we coded because the columns previously
# were strings, which is no longer indicated in the line of code above

# First check the age, which the survey says should range from 15 to 22
if(student['age'][(student['age'] < 15) & (student['age'] > 22)].shape[0] == 0):
    print("Age values are in range.")
else:
    print("At least one age value isn't in range.")
    
# Now check past class failures, which is apparently capped at 4 on the survey
if(student['failures'][(student['failures'] < 0) & (student['failures'] > 4)].shape[0] == 0):
    print("Failure values are in range.")
else:
    print("At least one failure value isn't in range.")
    
# Check fam_rel, which has a range of 1 - 5
if(student['fam_rel'][(student['fam_rel'] < 0) & (student['fam_rel'] > 4)].shape[0] == 0):
    print("Family relation values are in range.")
else:
    print("At least one family relation value isn't in range.")
    
# Check free_time, which has a range of 1 - 5
if(student['free_time'][(student['free_time'] < 0) & (student['free_time'] > 4)].shape[0] == 0):
    print("Free time values are in range.")
else:
    print("At least one free time value isn't in range.")
    
# Check hang_out, which has a range of 1 - 5
if(student['hang_out'][(student['hang_out'] < 0) & (student['hang_out'] > 4)].shape[0] == 0):
    print("Hang out values are in range.")
else:
    print("At least one hang out value isn't in range.")
        
# Check alcohol_frequency, which should have a range of 2 - 10
if(student['alcohol_frequency'][(student['alcohol_frequency'] < 2) & (student['alcohol_frequency'] > 10)].shape[0] == 0):
    print("Alcohol frequency values are in range.")
else:
    print("At least one alcohol frequency value isn't in range.")

# Check health, which has a range of 1 - 5
if(student['health'][(student['health'] < 0) & (student['health'] > 4)].shape[0] == 0):
    print("Health values are in range.")
else:
    print("At least one health value isn't in range.")
    
# Skip absences because we confirmed earlier that the values are in the range of 0 to 32, which is
# within the range of 0 - 93 indicated on the website

# Lastly, check final_grade, which should range from 0 to 20
if(student['final_grade'][(student['final_grade'] < 0) & (student['final_grade'] > 20)].shape[0] == 0):
    print("The final grade values are in range.")
else:
    print("At least one final grade value isn't in range.")

Age values are in range.
Failure values are in range.
Family relation values are in range.
Free time values are in range.
Hang out values are in range.
Alcohol frequency values are in range.
Health values are in range.
The final grade values are in range.


There seems to be no values from this survey that are out of range.

# Project Proposal (updated)

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/31  |  2 PM | Search for datasets that relate to our research question  | Go over the datasets and decide which would be best, what analysis we would like to do on them | 
| 2/7  |  2 PM |  Data cleaning assigned to each member | Assign roles for the project report (mainly coding, writing, analysis etc) | 
| 2/14  | 2 PM  | Brainstorm ideas for EDA, specifically what parts of the data we want to compare and how we will visualize them  | Review progress on EDA, discuss which analysis methods will work best |
| 2/21  | 2 PM  | Same as the previous row: continue brainstorming ideas for EDA | Review progress on project for the check-in  |
| 2/28  | 2 PM  | Start writing code based on our ideas from the previous two meetings | Share each other’s code and start figuring out what to put on Github |
| 3/7  | 2 PM  | Have code and analysis finished, begin writing conclusion and drafting video ideas | Discuss the video portion of the project in terms of editing and content, review  |
